In [196]:
import math
from pprint import pprint
from math import log, factorial
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
    permutations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter, xor
from bisect import bisect_right, bisect_left
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import sets
import matrix
import mathematics
from importlib import reload

for m in (arrays, trees, lists, graphs, search, strings, sets, matrix, mathematics):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import sets
import matrix as mt
import mathematics as m

print()

In [216]:
def smallest_intersecting_range(arrays: List[List[int]]) -> Tuple[int, int]:
    "Return the smallest range that intersects the sorted `arrays`."
    # The idea is to keep track of numbers from the `arrays` on the heap,
    # always increasing the lowest number with the next on the same array.
    # This approach tries to minimize the span of numbers on the heap.
    # The heap then always has the `min` and `max` values so far, which
    # allow us to determine the running smallest range.

    # Start with the lowest number in each array.
    h = [(a[0], a, int(1)) for a in arrays]
    heapify(h)  # make a min-heap out of the values

    # The minimum is always h[0], but we need to track the heap max.
    h_min, a, c = heappop(h)  # heap-minimum, array, next column
    h_max = max(h)[0]  # max value on the heap, so far
    # The range is stored as `(<length>, <start>)`,
    # which allows to easily run `min` on it.
    min_range = (h_max - h_min, h_min)  # minimum range, so far

    # Once we cannot add any more numbers from an array,
    # we abort, since otherwise the range would not include
    # numbers from that particular array and would be incorrect.
    while c < len(a):
        # `c` points to the next column.
        h_max = max(h_max, a[c])
        h_min, a, c = heappushpop(h, (a[c], a, c + 1))
        min_range = min(min_range, (h_max - h_min, h_min))

    return min_range[1], sum(min_range)

In [219]:
smallest_intersecting_range([[1, 3, 5], [7, 8, 9], [2, 4, 6], [2, 3, 8], [5, 7, 11]])

(5, 8)

In [220]:
smallest_intersecting_range([[1, 3, 5, 7, 9], [0, 2, 4, 6, 8], [2, 3, 5, 7, 11]])

(1, 2)